In [11]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from timeit import default_timer as timer

import sys
sys.path.append("..")

import helper.helper as h
import helper.methods as m

from notears.notears.notears import utils 
from notears.notears.notears import linear 

from importlib import reload  # Python 3.4+
reload(m)

<module 'helper.methods' from '..\\helper\\methods.py'>

In [23]:
utils.set_random_seed(1)

T, p, s0, graph_type, sem_type = 1000, 5, 5, 'ER', 'exp'
print(f"SEM based on a graph on {p} nodes and {s0} edges, {T} samples.")

B_true = utils.simulate_dag(p, s0, graph_type)
print(f"Original Adjacency Matrix:\n{B_true}.\n")

W_true = utils.simulate_parameter(B_true)
print(f"Original WAM:\n{np.round(W_true, 1)}.\n")

X = utils.simulate_linear_sem(W_true, T, sem_type)

X_norm = X - np.mean(X, axis=0, keepdims=True)

W_OMP, _ = m._K_OMP(X_norm, X_norm, normalize = True, F = [i * (p + 1) for i in range(p)])
W_OMP[np.abs(W_OMP) <= 0.30] = 0
W_est, h_val = linear.notears_linear(X, lambda1=0.1, loss_type='l2', verbose = False)
print(f"NOTEARS estimated WAM:\n{np.round(W_est, 1)}.\n")
print(f"OMP estimated WAM:\n{np.round(W_OMP, 1)}.\n")

acc = utils.count_accuracy(B_true, W_est != 0)
print(acc)

SEM based on a graph on 5 nodes and 5 edges, 1000 samples.
Original Adjacency Matrix:
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]].

Original WAM:
[[ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.5 -1.9]
 [-1.9  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   1.1]
 [ 0.   0.  -0.9  0.   0. ]].

NOTEARS estimated WAM:
[[ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.5 -1.7]
 [-1.9  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.9]
 [ 0.   0.  -0.8  0.   0. ]].

OMP estimated WAM:
[[ 0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.6 -1.9]
 [-1.9  0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   1.1]
 [ 0.   0.  -0.9  0.   0. ]].

{'fdr': 0.0, 'tpr': 1.0, 'fpr': 0.0, 'shd': 0, 'nnz': 5}
